In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv('correios.csv', error_bad_lines=False, sep=';', encoding='latin-1', decimal=",")

In [3]:
df.head()

,DATA,EMPRESA DE TRANSPORTES,CEP,DESTINO,PESO,CODIGO_AR,VALOR
0,19/05/2021,xx,11075-355,SP,27,JU785427856BR,1
1,19/05/2021,yy,30180-122,MG,23,JU785427873BR,1
2,21/05/2021,zz,32920-000,MG,78,JU785427887BR,1
3,21/05/2021,aa,35536-000,MG,85,JU785427895BR,1
4,21/05/2021,tt,32600-226,MG,32,JU785427900BR,1


In [4]:
# Função para transformar o CEP em um endereço 
from pycep_correios import get_address_from_cep, WebService
def get_address(cep):
    endereco = get_address_from_cep(cep, webservice=WebService.VIACEP)
    return endereco['logradouro'] + ", " + endereco['bairro'] + ", " + endereco['cidade'] + " - " + endereco['uf']

# Cria a coluna address com os endereços retornados a partir do CEP
df['Endereco'] = df['CEP'].apply(get_address)

In [5]:
df.head()

,DATA,EMPRESA DE TRANSPORTES,CEP,DESTINO,PESO,CODIGO_AR,VALOR,Endereco
0,19/05/2021,xx,11075-355,SP,27,JU785427856BR,1,"Avenida Doutor Bernardino de Campos, Vila Belm..."
1,19/05/2021,yy,30180-122,MG,23,JU785427873BR,1,"Avenida Olegário Maciel, Santo Agostinho, Belo..."
2,21/05/2021,zz,32920-000,MG,78,JU785427887BR,1,", , São Joaquim de Bicas - MG"
3,21/05/2021,aa,35536-000,MG,85,JU785427895BR,1,", , Piracema - MG"
4,21/05/2021,tt,32600-226,MG,32,JU785427900BR,1,"Avenida Presidente Kubitschek, Centro, Betim - MG"


In [6]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="GEO_END")

# Cria a coluna location com o local retornado a partir do endereço
df['Localizacao'] = df['Endereco'].apply(geolocator.geocode)
df['PontosLocalizacao'] = df['Localizacao'].apply(lambda loc: tuple(loc.point) if loc else None)


In [7]:
df.head()

,DATA,EMPRESA DE TRANSPORTES,CEP,DESTINO,PESO,CODIGO_AR,VALOR,Endereco,Localizacao,PontosLocalizacao
0,19/05/2021,xx,11075-355,SP,27,JU785427856BR,1,"Avenida Doutor Bernardino de Campos, Vila Belm...","(Avenida Doutor Bernardino de Campos, Vila Bel...","(-23.9503314, -46.3368361, 0.0)"
1,19/05/2021,yy,30180-122,MG,23,JU785427873BR,1,"Avenida Olegário Maciel, Santo Agostinho, Belo...","(Avenida Olegário Maciel, Santo Agostinho, Reg...","(-19.9244933, -43.9457195, 0.0)"
2,21/05/2021,zz,32920-000,MG,78,JU785427887BR,1,", , São Joaquim de Bicas - MG","(São Joaquim de Bicas, Região Geográfica Imedi...","(-20.0489052, -44.2731691, 0.0)"
3,21/05/2021,aa,35536-000,MG,85,JU785427895BR,1,", , Piracema - MG","(Piracema, Região Geográfica Imediata de Olive...","(-20.5091579, -44.4754464, 0.0)"
4,21/05/2021,tt,32600-226,MG,32,JU785427900BR,1,"Avenida Presidente Kubitschek, Centro, Betim - MG","(Avenida Juscelino Kubitscheck, Centro, Region...","(-19.9718077, -44.1972037, 0.0)"


In [10]:
#parei aqui
#type(df['Localizacao'][10].latitude)
#df = df.reindex(columns = df.columns.tolist() + ["Latitude","Longitude"])
#df['Latitude'] = df['Latitude'].astype(float, errors = 'raise')
#df['Longitude'] = df['Longitude'].astype(float, errors = 'raise')

In [ ]:
#for idx, _ in df.iterrows():
#    df['Latitude'].at[idx] = df['Localizacao'][idx].latitude

In [13]:
df['Endereco'][0]

'Avenida Doutor Bernardino de Campos, Vila Belmiro, Santos - SP'

### RASTREAMENTO

In [14]:
from pyrastreio import correios, jadlog, sequoia
df = df.assign(DATA="SEM INFO")
df = df.assign(HORA="SEM INFO")
df = df.assign(LOCAL="SEM INFO")
df = df.assign(MENSAGEM="SEM INFO")

In [15]:
x = 0
while x < len(df):
    rastreio= correios(df['CODIGO_AR'][x])[0]
    df['DATA'][x] = rastreio.get('data')
    df['HORA'][x] = rastreio.get('hora') 
    df['LOCAL'][x] = rastreio.get('local') 
    df['MENSAGEM'][x] = rastreio.get('mensagem') 
    x =x +1

<ipython-input-15-802c0782c23c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA'][x] = rastreio.get('data')
<ipython-input-15-802c0782c23c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HORA'][x] = rastreio.get('hora')
<ipython-input-15-802c0782c23c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LOCAL'][x] = rastreio.get('local')
<ipython-input-15-802c0782c23c>:7: SettingWithCopyWarning: 
A value is trying to be

In [19]:
df.head(2)

,DATA,EMPRESA DE TRANSPORTES,CEP,DESTINO,PESO,CODIGO_AR,VALOR,Endereco,Localizacao,PontosLocalizacao,HORA,LOCAL,MENSAGEM
0,24/05/2021,xx,11075-355,SP,27,JU785427856BR,1,"Avenida Doutor Bernardino de Campos, Vila Belm...","(Avenida Doutor Bernardino de Campos, Vila Bel...","(-23.9503314, -46.3368361, 0.0)",11:52,SANTOS / SP,Objeto entregue ao destinatário
1,21/05/2021,yy,30180-122,MG,23,JU785427873BR,1,"Avenida Olegário Maciel, Santo Agostinho, Belo...","(Avenida Olegário Maciel, Santo Agostinho, Reg...","(-19.9244933, -43.9457195, 0.0)",11:30,BELO HORIZONTE / MG,Objeto entregue ao destinatário
